In [ ]:
from utils import img_util
from utils import hed_util
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

MODEL_NAME = 'model'
HED_NAME = 'HED'
# Grab path to current working directory
CWD_PATH = os.getcwd()
HED_PROTOTEXT_PATH = os.path.join(CWD_PATH,MODEL_NAME,HED_NAME,'deploy.prototxt')
HED_CAFFEE_MODEL_PATH = os.path.join(CWD_PATH,MODEL_NAME,HED_NAME,'hed_pretrained_bsds.caffemodel')

TEST_IMAGE_PATH = os.path.join(CWD_PATH, 'test_images')


net = hed_util.load_dnn(HED_PROTOTEXT_PATH,HED_CAFFEE_MODEL_PATH)

In [ ]:
def hed_edge(net, image):   
    hed_edge = hed_util.edge_detection(net, image)
    return cv2.cvtColor(hed_edge,cv2.COLOR_BGR2GRAY)  

In [ ]:
#img = cv2.imread(PATH_TO_IMAGE, cv2.COLOR_BGR2RGB)
PATH_TO_IMAGE = os.path.join(TEST_IMAGE_PATH, '001457.jpeg')
img = plt.imread(PATH_TO_IMAGE)
gray = hed_edge(net, img)
binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
cnts = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
boxes = []
cnt = None
max_area = x = y = w = h = 0
i=0
print('initial max area ', max_area)
for c in cnts:
    area = cv2.contourArea(c)
    if area > max_area:
        #print('{} > {}'.format(area, max_area))
        x, y, w, h = cv2.boundingRect(c)
        max_area = area
        cnt = c    
        boxes.append(c)
        i+=1

plt.imshow(img)
print('number of contour ',i)

In [ ]:

boxes = np.asarray(boxes)
box = boxes[len(boxes)-1]
#box = boxes[0]
x1,y1,w1,h1 = cv2.boundingRect(box)
print('x1 {} y1 {} w1 {} h1 {} x+w {} y+h {}'.format(x1,y1,w1,h1,x+w,y+h))

#cv2.drawContours(img, [box], 0, (0,  255, 0), 3)
#cv2.rectangle(img, (x1,y1), (x1+w1, y1+h1), (255, 0, 0), 3)
#cv2.drawContours(img, [cnt], 0, (0,  255, 0), 3)
#cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 0), 3)
#cv2.rectangle(img, (left,top), (right,bottom), (255, 0, 0), 2)
print('x {} y {} w {} h {}'.format(x1,y1,w1,h1))
crop_img = img[y1:y1+h1, x1:x1+w1]
plt.imshow(crop_img)